In [1]:
import pandas as pd
import re
import requests
import urllib.parse

In [5]:
# lat/lon de France métropolitaine + Corse
fr_box=[-7.382813,41.129021,11.403809,51.082822]

In [12]:
# crée un dictionnaire aéroport - code administratif de la ville associée
gares=pd.read_csv("gares.csv", dtype={'gare code': str},index_col="aéroport")
code_dict=gares["gare code"].to_dict()

In [17]:
# import des vols 
df=pd.read_csv("flights.csv")
# turn : into 0
df[df==":"]=0
# remove nan
df=df[:-3]

In [2]:
def obtenir_coord(lieu):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(lieu) +'?format=json'
    response = requests.get(url).json()
    if len(response)==0:
        return None
    return response[0]["lat"],response[0]["lon"]

In [7]:
# cf l'API de la SNCF et Navitia.io pour la requête
def temps_trajet(code_depart,code_arrivee):
    url='https://api.sncf.com/v1/coverage/sncf/journeys?from=admin%3Afr%3A{0}&to=admin%3Afr%3A{1}&datetime=20210420T000000&datetime_represents=departure&timeframe_duration=86400&'.format(code_depart,code_arrivee)
    params = dict(key='31004076-db95-46fd-ba09-198d4e845d0b')
    res = requests.get(url, params=params).json()
    # si pas de trajet retourne un temps 'infini'
    try :
        trajets=res['journeys']
    except KeyError:
        return 60*10000
    # on prend le trajet le plus court de la journée
    temps_min=min([trajet['duration'] for trajet in trajets])
    return temps_min

In [4]:
def in_box(box,lat,lon):
    lon=float(lon)
    lat=float(lat)
    return (lon>box[0])&(lat>box[1])&(lon<box[2])&(lat<box[3])

In [11]:
links=[]
nrow=df.shape[0]

for i in range(nrow):
    
    # sélection des vols intérieurs (dont DOM-TOM)
    if not re.search(r"FR\_\w+\_FR\w+",df["CODE"][i]):
                continue
            
    # avant 2001 pas de distinction entre Roissy et Orly, juste Airport System, Paris Airport      
    vol=re.sub("airport SYSTEM - ","",df["PAIR"][i])
    vol=re.sub("airport","",vol)
    depart,arrivee=re.split(' - ', vol)
    depart=depart[:-1]
    arrivee=arrivee[:-1]
    
    # ajout des coordonnées si trouvés
    # affiche les vols dont départ ou arrivée ne sont pas trouvés
    try :
        lat_d,lon_d=obtenir_coord(depart)
        lat_a,lon_a=obtenir_coord(arrivee)
    except TypeError: 
        print(arrivee,depart)
        continue
        
    # garde que France métropolitaine + Corse
    if not in_box(fr_box,lat_d,lon_d) or not in_box(fr_box,lat_a,lon_a):
        continue

    # ajout du temps de parcours en train
    temps=temps_trajet(code_dict[depart],code_dict[arrivee])
    
    # création du dictionnaire
    d={}
    d["lat_d"]=lat_d
    d["lon_d"]=lon_d
    d["lat_a"]=lat_a
    d["lon_a"]=lon_a
    d["CODE"]=df["CODE"][i]
    d["depart"]=depart
    d["arrivee"]=arrivee
    d["temps"]=temps
    links.append(d)

PARIS-ORLY BERGERAC-ROUMANIERE
MARIPASO CAYENNE-ROCHAMBEAU
SAINT MARTIN, GRAND CASE, GUADELOUPE AIME CESAIRE, MARTINIQUE
AIME CESAIRE, MARTINIQUE SAINT MARTIN, GRAND CASE, GUADELOUPE
SAINT BARTHELEMY,FRENCH ANTILLES SAINT MARTIN, GRAND CASE, GUADELOUPE
POINTE-A-PITRE,LE RAIZET, GUADELOUPE SAINT MARTIN, GRAND CASE, GUADELOUPE
SAINT MARTIN, GRAND CASE, GUADELOUPE SAINT BARTHELEMY,FRENCH ANTILLES
POINTE-A-PITRE,LE RAIZET, GUADELOUPE SAINT BARTHELEMY,FRENCH ANTILLES
SAINT MARTIN, GRAND CASE, GUADELOUPE POINTE-A-PITRE,LE RAIZET, GUADELOUPE
SAINT BARTHELEMY,FRENCH ANTILLES POINTE-A-PITRE,LE RAIZET, GUADELOUPE


In [12]:
links_df=pd.DataFrame(links)
links_df=links_df.merge(df,how='left',left_on='CODE',right_on='CODE')

In [14]:
links_df.to_csv("vols.csv")